# Wolverine Glacier Model
## Rewritten using geopandas 
## Weird kernal errors happening

In [ ]:
import firedrake
import icepack
import geojson
import xarray 

In [ ]:
import geopandas as gpd

outline_filename = "wolv_tr_dem.geojson"

In [ ]:
dataframe = gpd.read_file(outline_filename)
corrected_df = dataframe.to_crs(32606)

corrected_df

In [ ]:
outline_lat_lon = corrected_df.geometry
utm_crs = outline_lat_lon.estimate_utm_crs()

outline_utm = outline_lat_lon.to_crs(utm_crs)
outline_json = geojson.loads(outline_utm.to_json())

outline = geojson.utils.map_tuples(lambda x: x[:2], outline_json)

geometry = icepack.meshing.collection_to_geo(outline)
geo_filename = "wolverine.geo"

with open(geo_filename, "w") as geo_file:
    geo_file.write(geometry.get_code())

In [ ]:
import numpy as np
import icepack.plot

fig, axes = icepack.plot.subplots()

for feature in outline["features"]:
    for line_string in feature["geometry"]["coordinates"]:
        xs = np.array(line_string)
        axes.plot(xs[:, 0], xs[:, 1], linewidth=2)

In [ ]:
import subprocess
import pathlib

path = pathlib.Path("wolverine.msh")
if not path.exists():
    command = "gmsh -2 -v 2 -o wolverine.msh wolverine.geo"
    subprocess.run(command.split())

In [ ]:
!gmsh -2 -format msh2 -v 2 -o wolverine.msh wolverine.geo

In [ ]:
mesh = firedrake.Mesh("wolverine.msh")

In [ ]:
import icepack.plot

fig, axes = icepack.plot.subplots()
firedrake.triplot(mesh, axes=axes)
#axes.legend()

In [ ]:
Q = firedrake.FunctionSpace(mesh, "CG", 2)
V = firedrake.VectorFunctionSpace(mesh, "CG", 2)

## Input Data
> Thickness from Farinotti
> 
> Velocity from ITS_LIVE

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import rasterio

thickness_fn = 'wolv_thickness.tif'
thickness_ds = rasterio.open(thickness_fn)

bounds = thickness_ds.bounds
left, right, bottom, top = bounds.left, bounds.right, bounds.bottom, bounds.top
extent = (left, right, bottom, top)

fig, axes = plt.subplots()
axes.imshow(thickness_ds.read(indexes=1), extent=extent)
firedrake.triplot(mesh, interior_kw={"linewidth": 0.02}, axes=axes);

In [ ]:
thickness_ds.crs

In [ ]:
from rasterio.windows import from_bounds

with rasterio.open(thickness_fn) as tif:
    print(tif.transform)
    #transform = tif.transform
tif.crs

In [ ]:
h0 = icepack.interpolate(thickness_ds, Q)

In [ ]:
!curl -O -C - https://its-live-data.s3.amazonaws.com/velocity_mosaic/landsat/v00.0/static/cog/ALA_G0120_0000_vx.tif
!curl -O -C - https://its-live-data.s3.amazonaws.com/velocity_mosaic/landsat/v00.0/static/cog/ALA_G0120_0000_vy.tif

In [ ]:
!rio warp ALA_G0120_0000_vx.tif vx.tif --dst-crs EPSG:32606
!rio warp ALA_G0120_0000_vy.tif vy.tif --dst-crs EPSG:32606

In [ ]:
import matplotlib.pyplot as plt

vx_fn = 'vx.tif'
vx_file = rasterio.open(vx_fn)

vy_fn = 'vy.tif'
vy_file = rasterio.open(vy_fn)

vx_file.crs

In [ ]:
transform = vx_file.transform
window = rasterio.windows.from_bounds(
    left=bounds.left, bottom=bounds.bottom, right=bounds.right, top=bounds.top, transform=transform
)

In [ ]:
vx = vx_file.read(indexes=1, window=window, masked=True)
vy = vy_file.read(indexes=1, window=window, masked=True)

In [ ]:
fig, axes = plt.subplots()
axes.imshow(vy, extent=extent)
firedrake.triplot(mesh, axes=axes);

In [ ]:
u_raw = icepack.interpolate((vx_file, vy_file), V)

We can see from the quiver plot that the directions are rotated by $\pi/2$ clockwise from what they should be.
This is a consequence of the coordinate reprojection that we did from the original polar stereographic projection of the raw data to the UTM projection that we're using here.

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect("equal")
arrows = firedrake.quiver(u_raw, axes=axes)
fig.colorbar(arrows, label="meters/year");

This looks much more sensible.

In [ ]:
u = firedrake.interpolate(firedrake.as_vector((-u_raw[1], u_raw[0])), V)

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect("equal")
arrows = firedrake.quiver(u, axes=axes)
fig.colorbar(arrows, label="meters/year");